## Imports

In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup as BS
import cloudscraper
import os.path
import json
import csv

## Get airports and check which ones need different names

In [46]:
with open('utils/airports.csv','r',encoding='utf-8-sig') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\n')
    airports = [x[0] for x in reader]

## Get connecting airports

In [36]:
# We want to webscrape urls that look like this:
# https://www.flightsfrom.com/ATL/destinations?countries=US&durationFrom=44&durationTo=999
scraper = cloudscraper.create_scraper()
connections = {}
if not os.path.isfile("utils/outgoing.json"):
    for airport in airports:
        url = "https://www.flightsfrom.com/" + airport + "/destinations?countries=US&durationFrom=1&durationTo=999"
        r = scraper.get(url)
        soup = BS(r.text, 'html.parser')
        outgoingAirports = soup.find_all(class_ = "airport-font-midheader destination-search-item")
        connections[airport] = []
        for outgoing in outgoingAirports:
            connections[airport].append(str(outgoing).split(" ")[-1][:3])
        if len(connections[airport]) == 0:
            print(airport)
            del connections[airport]
    json.dump(connections,open("utils/outgoing.json","x"))
else:
    connections = json.load(open("utils/outgoing.json"))

## Create Airport Integer Encodings

In [39]:
airport_to_int = {}
for i,airport in enumerate(airports):
    airport_to_int[airport] = i
json.dump(airport_to_int,open("utils/airportEncodings.json","x"))

## Create links to all flight searches

In [37]:
#We want to create urls that look like this:
# "https://www.google.com/travel/flights?q=one+way+flights+from+BLANK+to+BLANK+on+2023-06-09"
bad = json.load(open('utils/nameSubstitution.json'))
if not os.path.isfile("utils/flightLinks.csv"):
    f = open("utils/flightLinks.csv","x")
    for startAirport in connections.keys():
        for endAirport in connections[startAirport]:
            if startAirport in bad: startAirport = bad[startAirport]
            if endAirport in bad: endAirport = bad[endAirport]
            searchURL = "https://www.google.com/travel/flights?q=one+way+flight+from+"+startAirport+"+to+"+endAirport+"+on+"
            f.write(searchURL+"\n")
    f.close()